In [1]:
from itertools import combinations
import numpy as np
from z3 import *
import utils

In [2]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), And(at_most_one_np(bool_vars, name)))

In [3]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)] # Need the s to have unique names
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))

    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(s[i - 1]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i - 1])))
    
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_most_one_seq(bool_vars, name), at_least_one_seq(bool_vars))

In [18]:
def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name), at_least_k_seq(bool_vars, k, name))

In [19]:
WIDTH = 0
HEIGHT = 1

In [23]:
def vlsi_design(w, n, dims):
    s = Solver()
    upper_bound = sum(dims[:, 1])
    print("Upper bound:", upper_bound)

    # Model the problem
    v = [[[Bool(f"v_{x}_{y}_{c}") for c in range(n)] for y in range(upper_bound)] for x in range(w)]
    
    # At most one circuit in each grid cell
    for x in range(w):
        for y in range(upper_bound):
            predicate = v[x][y]
            # print(predicate)
            s.add(at_most_one_seq(predicate, f"valid_cell_{x}_{y}"))
    
    # All the circuits must be placed in the grid
    for c in range(n):
        circuit_area = dims[c, WIDTH] * dims[c, HEIGHT]
        print(circuit_area)
        predicate = [v[x][y][c] for x in range(w) for y in range(upper_bound)]
        print(predicate)
        s.add(exactly_k_seq([v[x][y][c] for x in range(w) for y in range(upper_bound)], circuit_area, f"valid_placement_{c}"))
    
    
    if s.check() == sat:
        model = s.model()
        sol = []
        for x in range(w):
            sol.append([])
            for y in range(upper_bound):
                found_circuit = False
                for c in range(n):
                    if model.evaluate(v[x][y][c]):
                        found_circuit = True
                        sol[x].append(c+1)
                if not found_circuit:
                    sol[x].append(0)

        return sol
    else:
        print("Buhu")

In [24]:
w = 8
n = 4
dims = np.array([[3, 3], [3, 5], [5, 3], [5, 5]])

sol = vlsi_design(w, n, dims)
utils.display_sudoku(sol, w, len(sol))

Upper bound: 16
9
[v_0_0_0, v_0_1_0, v_0_2_0, v_0_3_0, v_0_4_0, v_0_5_0, v_0_6_0, v_0_7_0, v_0_8_0, v_0_9_0, v_0_10_0, v_0_11_0, v_0_12_0, v_0_13_0, v_0_14_0, v_0_15_0, v_1_0_0, v_1_1_0, v_1_2_0, v_1_3_0, v_1_4_0, v_1_5_0, v_1_6_0, v_1_7_0, v_1_8_0, v_1_9_0, v_1_10_0, v_1_11_0, v_1_12_0, v_1_13_0, v_1_14_0, v_1_15_0, v_2_0_0, v_2_1_0, v_2_2_0, v_2_3_0, v_2_4_0, v_2_5_0, v_2_6_0, v_2_7_0, v_2_8_0, v_2_9_0, v_2_10_0, v_2_11_0, v_2_12_0, v_2_13_0, v_2_14_0, v_2_15_0, v_3_0_0, v_3_1_0, v_3_2_0, v_3_3_0, v_3_4_0, v_3_5_0, v_3_6_0, v_3_7_0, v_3_8_0, v_3_9_0, v_3_10_0, v_3_11_0, v_3_12_0, v_3_13_0, v_3_14_0, v_3_15_0, v_4_0_0, v_4_1_0, v_4_2_0, v_4_3_0, v_4_4_0, v_4_5_0, v_4_6_0, v_4_7_0, v_4_8_0, v_4_9_0, v_4_10_0, v_4_11_0, v_4_12_0, v_4_13_0, v_4_14_0, v_4_15_0, v_5_0_0, v_5_1_0, v_5_2_0, v_5_3_0, v_5_4_0, v_5_5_0, v_5_6_0, v_5_7_0, v_5_8_0, v_5_9_0, v_5_10_0, v_5_11_0, v_5_12_0, v_5_13_0, v_5_14_0, v_5_15_0, v_6_0_0, v_6_1_0, v_6_2_0, v_6_3_0, v_6_4_0, v_6_5_0, v_6_6_0, v_6_7_0, v_6_8_0, 

TypeError: object of type 'NoneType' has no len()